In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


 # X To A Summary

In [1]:
import os
import numpy as np
import pandas as pd
from larval_gonad.io import pickle_load, shelve_load, feather_to_cluster_matrix


In [2]:
pd.options.display.max_rows = 200

try:
    os.chdir(os.path.join(os.getcwd(), 'docs'))
    print(os.getcwd())
except:
    pass


/var/home/fearjm/Projects/larval_gonad/docs


In [3]:
fbgn2chrom = (
    pd.read_feather("../references/gene_annotation_dmel_r6-26.feather")
    .set_index("FBgn")
    .FB_chrom
    .rename("chrom")
)


In [4]:
expressed = pickle_load("../output/cellselection-wf/expressed_genes.pkl")
print(f"Number of expressed genes: {len(expressed):,}")

# Expressed by chrom
fbgn2chrom.reindex(expressed).value_counts()


Number of expressed genes: 14,347


3R                                 3467
3L                                 2867
2R                                 2861
2L                                 2767
X                                  2207
4                                   102
Y                                    42
mitochondrion_genome                 18
rDNA                                  5
Unmapped_Scaffold_8_D1580_D1567       2
211000022279392                       1
211000022280341                       1
211000022279264                       1
211000022280328                       1
211000022278436                       1
211000022278449                       1
211000022279188                       1
211000022278760                       1
211000022279165                       1
Name: chrom, dtype: int64

In [5]:
commonly_expressed = pickle_load("../output/cellselection-wf/commonly_expressed_genes.pkl")
print(f"Number of commonly expressed genes: {len(commonly_expressed):,}")

# Commonly expressed by chrom
fbgn2chrom.reindex(commonly_expressed).value_counts()


Number of commonly expressed genes: 589


3R                      145
2R                      138
2L                      126
3L                      101
X                        65
mitochondrion_genome     11
4                         3
Name: chrom, dtype: int64

In [6]:
# Y genes in other clusters
y_genes = fbgn2chrom[fbgn2chrom == "Y"].index.tolist()
tpm = feather_to_cluster_matrix("../output/seurat3-cluster-wf/tpm_by_cluster.feather").reindex(y_genes)
_num = tpm[(tpm.iloc[:, 4:] > 0).any(axis=1)].shape[0]
print(f"Number of Y genes in somatic clusters: {_num:,}")


Number of Y genes in somatic clusters: 35


In [7]:
cb = shelve_load("../output/x-to-a-wf/db/commonly_expressed.bak")


In [8]:
# Median Ratios
cb['data'].groupby(['cluster', 'ratio_type']).ratio.median().unstack().dropna(axis=1)


ratio_type,fourth_to_a_ratio,x_to_a_ratio
cluster,,
G,1.500569,1.081121
EPS,0.841584,0.859186
MPS,0.487106,0.646613
LPS,0.331061,0.560440
C1,0.883034,1.208765
C2,1.383563,1.382147
C3,1.021378,1.231345
C4,1.595174,1.386552
T,1.210826,1.387989


In [9]:
# Log2FC
np.log2(cb['data'].groupby(['cluster', 'ratio_type']).ratio.median().unstack().dropna(axis=1))


ratio_type,fourth_to_a_ratio,x_to_a_ratio
cluster,,
G,0.585510,0.112528
EPS,-0.248821,-0.218957
MPS,-1.037692,-0.629026
LPS,-1.594830,-0.835369
C1,-0.179459,0.273533
C2,0.468388,0.466911
C3,0.030518,0.300235
C4,0.673714,0.471501
T,0.275992,0.472996
